# Installs

In [ ]:
!pip install diffusers gradio openai==0.28

# Model Setup

In [ ]:
from diffusers import StableDiffusion3Pipeline
import torch
import transformers
import gradio as gr
from huggingface_hub import login, InferenceClient
import openai

openai.api_key = 'OPENAI_API_KEY'
login(token='HF_ACCESS_KEY')

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", text_encoder_3=None, tokenizer_3=None,  torch_dtype=torch.float16).to('cuda')

def generate_image_from_text(text):
    image = pipe(text, num_inference_steps=28).images[0]
    return image


def create_storybook(topic):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a great story writer. You write creative and intriguing story based on the given topic. Your sentences will be used to generate a story book so create informative sentences."},
            {"role": "user", "content": f"Generate a short story on {topic}."},
        ]
    )
    story = response['choices'][0]['message']['content']
    # story = response.choices[0].message.content
    story_parts = story.split(". ")

    images = []
    for part in story_parts:
        if part.strip():  # Avoid empty strings
            image = generate_image_from_text(part)
            images.append(image)

    return story, images

def interface_phase():
  gr.Interface(fn = create_storybook, inputs="text", outputs=["text","gallery"]).launch(debug = True)

if __name__ == "__main__":
    interface_phase()